In [1]:
import pandas as pd
import numpy as np
from psruq.source.table_utils import (
    collect_scores_into_dict,
    extract_same_different_dataframes,
    ood_detection_pairs_,
    aggregate_over_measures,
)
from psruq.source.path_config import REPOSITORY_ROOT
from source.metrics.constants import GName
from source.losses.constants import LossName
from source.datasets.constants import DatasetName
from source.models.constants import ModelSource
from IPython.display import display

pd.set_option("display.max_rows", None)

stty: 'standard input': Inappropriate ioctl for device


In [6]:
ind_dataset_name = DatasetName.CIFAR10_NOISY_LABEL.value
model_source = ModelSource.OUR_MODELS.value

# full_ood_rocauc = pd.read_pickle(
#     f"{REPOSITORY_ROOT}/tables/central_tables/final/{ind_dataset_name}_{model_source}_full_ood_rocauc.pkl"
# )

In [7]:
### FOR MISCLASSIFICATION

full_ood_rocauc = pd.read_pickle(
    f"{REPOSITORY_ROOT}/tables/central_tables/final/{ind_dataset_name}_{model_source}_full_mis_rocauc.pkl"
)

full_ood_rocauc["Dataset"] = ind_dataset_name

In [8]:
unique_uq_measures = set([' '.join(el.split(' ')[1:]) for el in full_ood_rocauc.UQMetric.unique()])

In [9]:
full_ood_rocauc = full_ood_rocauc[~((full_ood_rocauc.UQMetric.str.find('central') > -1) & (full_ood_rocauc.base_rule == GName.ZERO_ONE_SCORE.value))] # because Central not defined for Zero-One score
full_ood_rocauc = full_ood_rocauc[~((full_ood_rocauc.UQMetric.str.find('central') > -1) & (full_ood_rocauc.UQMetric.str.find('inner') > -1) & (full_ood_rocauc.base_rule == GName.BRIER_SCORE.value) & (full_ood_rocauc.UQMetric.str.find('ExcessRisk') > -1))] # because Central coincides with Inner for Brier

full_ood_rocauc = full_ood_rocauc[full_ood_rocauc.LossFunction == LossName.CROSS_ENTROPY.value] # only specific loss

In [10]:
full_ood_rocauc.columns

Index(['UQMetric', 'LossFunction', 'RocAucScores_array', 'architecture',
       'training_dataset', 'base_rule', 'RiskType', 'Dataset'],
      dtype='object')

In [11]:
# full_ood_rocauc.UQMetric.unique()

In [12]:
df_coinciding = None

mapping_loss_to_g = {
    LossName.CROSS_ENTROPY.value: GName.LOG_SCORE.value,
    LossName.BRIER_SCORE.value: GName.BRIER_SCORE.value,
    LossName.SPHERICAL_SCORE.value: GName.SPHERICAL_SCORE.value,
}


for loss_ in LossName:
    loss = loss_.value
    if df_coinciding is None:
        df_coinciding = full_ood_rocauc[(full_ood_rocauc.LossFunction == loss) & (full_ood_rocauc.base_rule == mapping_loss_to_g[loss])]
    else:
        df_coinciding = pd.concat([df_coinciding, full_ood_rocauc[(full_ood_rocauc.LossFunction == loss) & (full_ood_rocauc.base_rule == mapping_loss_to_g[loss])]])

In [13]:
diff_elements = [i for i in np.array(full_ood_rocauc.index) if i not in np.array(df_coinciding.index)]

In [14]:
df_diff = full_ood_rocauc.loc[diff_elements]

In [15]:
def mean_riskwise(dataframe):
    
    measure_dict = {}
    for dataset_ in full_ood_rocauc.Dataset.unique():
        measure_dict[dataset_] = {}
        for uq_measure in unique_uq_measures:
            vals = dataframe[dataframe.UQMetric.str.endswith(uq_measure) & (dataframe.Dataset == dataset_)]
            
            if (uq_measure.find('energy') > -1) & (len(vals) == 0):
                continue
    
            measure_dict[dataset_][uq_measure] = {}
    
            mean_ = np.vstack(vals.RocAucScores_array.values).mean()
    
            measure_dict[dataset_][uq_measure] = mean_
    return measure_dict

In [16]:
full_stratified = pd.DataFrame.from_dict(mean_riskwise(full_ood_rocauc.copy())).sort_index()
full_stratified

,cifar10_noisy_label
BayesRisk central,0.793217
BayesRisk inner,0.796328
BayesRisk outer,0.765180
ExcessRisk central central,0.500000
ExcessRisk central inner,0.666374
ExcessRisk central outer,0.720505
ExcessRisk inner central,0.655247
ExcessRisk inner inner,0.500000
ExcessRisk inner outer,0.714054
ExcessRisk outer central,0.710383


In [17]:
coinciding_stratified = pd.DataFrame.from_dict(mean_riskwise(df_coinciding.copy())).sort_index()
coinciding_stratified

,cifar10_noisy_label
BayesRisk central,0.804374
BayesRisk inner,0.783553
BayesRisk outer,0.782573
ExcessRisk central central,0.500000
ExcessRisk central inner,0.612817
ExcessRisk central outer,0.699839
ExcessRisk inner central,0.592383
ExcessRisk inner inner,0.500000
ExcessRisk inner outer,0.676975
ExcessRisk outer central,0.668474


In [18]:
different_stratified = pd.DataFrame.from_dict(mean_riskwise(df_diff.copy())).sort_index()
different_stratified

,cifar10_noisy_label
BayesRisk central,0.787639
BayesRisk inner,0.800586
BayesRisk outer,0.759383
ExcessRisk central central,0.500000
ExcessRisk central inner,0.719932
ExcessRisk central outer,0.730838
ExcessRisk inner central,0.718112
ExcessRisk inner inner,0.500000
ExcessRisk inner outer,0.726413
ExcessRisk outer central,0.731337


In [19]:
100 * (coinciding_stratified - full_stratified) / full_stratified

,cifar10_noisy_label
BayesRisk central,1.406561
BayesRisk inner,-1.604175
BayesRisk outer,2.272977
ExcessRisk central central,0.000000
ExcessRisk central inner,-8.037112
ExcessRisk central outer,-2.868182
ExcessRisk inner central,-9.594064
ExcessRisk inner inner,0.000000
ExcessRisk inner outer,-5.192693
ExcessRisk outer central,-5.899430


In [20]:
100 * (different_stratified - full_stratified) / full_stratified

,cifar10_noisy_label
BayesRisk central,-0.703280
BayesRisk inner,0.534725
BayesRisk outer,-0.757659
ExcessRisk central central,0.000000
ExcessRisk central inner,8.037112
ExcessRisk central outer,1.434091
ExcessRisk inner central,9.594064
ExcessRisk inner inner,0.000000
ExcessRisk inner outer,1.730898
ExcessRisk outer central,2.949715


In [21]:
relative_tab =  100 * (coinciding_stratified - different_stratified) / different_stratified

relative_tab

,cifar10_noisy_label
BayesRisk central,2.124785
BayesRisk inner,-2.127523
BayesRisk outer,3.053773
ExcessRisk central central,0.000000
ExcessRisk central inner,-14.878428
ExcessRisk central outer,-4.241447
ExcessRisk inner central,-17.508364
ExcessRisk inner inner,0.000000
ExcessRisk inner outer,-6.805790
ExcessRisk outer central,-8.595599


In [22]:
# relative_tab = relative_tab.drop(columns=[DatasetName.CIFAR10_NOISY_LABEL.value, DatasetName.CIFAR10.value])
# relative_tab = relative_tab.drop(columns=[DatasetName.TINY_IMAGENET.value])

In [23]:


# Merge the mean and std tables based on their index to create a combined LaTeX table.
merged_table = relative_tab.round(3).copy()

# For each numeric column, combine the mean and std in the format: mean \pm std
for col in relative_tab.columns:
    merged_table[col] = relative_tab.round(3)[col].astype(str) #+ " $\\pm$ " + full_std_tab[col].round(3).astype(str)

# Create LaTeX format
latex_table = merged_table.to_latex(index=True, escape=False, float_format="%.2f")

# Output the resulting LaTeX table for the user
print(latex_table)

\begin{tabular}{ll}
\toprule
 & cifar10_noisy_label \\
\midrule
BayesRisk central & 2.125 \\
BayesRisk inner & -2.128 \\
BayesRisk outer & 3.054 \\
ExcessRisk central central & 0.0 \\
ExcessRisk central inner & -14.878 \\
ExcessRisk central outer & -4.241 \\
ExcessRisk inner central & -17.508 \\
ExcessRisk inner inner & 0.0 \\
ExcessRisk inner outer & -6.806 \\
ExcessRisk outer central & -8.596 \\
ExcessRisk outer inner & -7.016 \\
ExcessRisk outer outer & -7.848 \\
TotalRisk central central & 2.125 \\
TotalRisk central inner & 1.97 \\
TotalRisk central outer & 1.609 \\
TotalRisk inner central & -4.154 \\
TotalRisk inner inner & -2.128 \\
TotalRisk inner outer & -5.188 \\
TotalRisk outer central & -4.155 \\
TotalRisk outer inner & -2.128 \\
TotalRisk outer outer & -5.188 \\
energy inner & nan \\
energy outer & nan \\
\bottomrule
\end{tabular}



In [24]:
replace_dictionary = {
    "BayesRisk central": r"\(\Rtildebayes^{(3)}\)",
    "BayesRisk inner": r"\(\Rtildebayes^{(2)}\)",
    "BayesRisk outer": r"\(\Rtildebayes^{(1)}\)",
    "ExcessRisk central central": r"\(\Rtildeexc^{(3, 3)}\)",
    "ExcessRisk central inner": r"\(\Rtildeexc^{(3, 2)}\)",
    "ExcessRisk central outer": r"\(\Rtildeexc^{(3, 1)}\)",
    "ExcessRisk inner central": r"\(\Rtildeexc^{(2, 3)}\)",
    "ExcessRisk inner inner": r"\(\Rtildeexc^{(2, 2)}\)",
    "ExcessRisk inner outer": r"\(\Rtildeexc^{(2, 1)}\)",
    "ExcessRisk outer central": r"\(\Rtildeexc^{(1, 3)}\)",
    "ExcessRisk outer inner": r"\(\Rtildeexc^{(1, 2)}\)",
    "ExcessRisk outer outer": r"\(\Rtildeexc^{(1, 1)}\)",
    "TotalRisk central central": r"\(\Rtildetot^{(3, 3)}\)",
    "TotalRisk central inner": r"\(\Rtildetot^{(3, 2)}\)",
    "TotalRisk central outer": r"\(\Rtildetot^{(3, 1)}\)",
    "TotalRisk inner central": r"\(\Rtildetot^{(2, 3)}\)",
    "TotalRisk inner inner": r"\(\Rtildetot^{(2, 2)}\)",
    "TotalRisk inner outer": r"\(\Rtildetot^{(2, 1)}\)",
    "TotalRisk outer central": r"\(\Rtildetot^{(1, 3)}\)",
    "TotalRisk outer inner": r"\(\Rtildetot^{(1, 2)}\)",
    "TotalRisk outer outer": r"\(\Rtildetot^{(1, 1)}\)",
    "energy inner": r"\( E(x;\E_{\theta}f_{\theta}) \)",
    "energy outer": r"\( \E_{\theta} E(x;f_{\theta}) \)",
    
} 
for key in replace_dictionary.keys():
    latex_table = latex_table.replace(key, replace_dictionary[key])

print(latex_table)

\begin{tabular}{ll}
\toprule
 & cifar10_noisy_label \\
\midrule
\(\Rtildebayes^{(3)}\) & 2.125 \\
\(\Rtildebayes^{(2)}\) & -2.128 \\
\(\Rtildebayes^{(1)}\) & 3.054 \\
\(\Rtildeexc^{(3, 3)}\) & 0.0 \\
\(\Rtildeexc^{(3, 2)}\) & -14.878 \\
\(\Rtildeexc^{(3, 1)}\) & -4.241 \\
\(\Rtildeexc^{(2, 3)}\) & -17.508 \\
\(\Rtildeexc^{(2, 2)}\) & 0.0 \\
\(\Rtildeexc^{(2, 1)}\) & -6.806 \\
\(\Rtildeexc^{(1, 3)}\) & -8.596 \\
\(\Rtildeexc^{(1, 2)}\) & -7.016 \\
\(\Rtildeexc^{(1, 1)}\) & -7.848 \\
\(\Rtildetot^{(3, 3)}\) & 2.125 \\
\(\Rtildetot^{(3, 2)}\) & 1.97 \\
\(\Rtildetot^{(3, 1)}\) & 1.609 \\
\(\Rtildetot^{(2, 3)}\) & -4.154 \\
\(\Rtildetot^{(2, 2)}\) & -2.128 \\
\(\Rtildetot^{(2, 1)}\) & -5.188 \\
\(\Rtildetot^{(1, 3)}\) & -4.155 \\
\(\Rtildetot^{(1, 2)}\) & -2.128 \\
\(\Rtildetot^{(1, 1)}\) & -5.188 \\
\( E(x;\E_{\theta}f_{\theta}) \) & nan \\
\( \E_{\theta} E(x;f_{\theta}) \) & nan \\
\bottomrule
\end{tabular}



In [17]:
# Cross Entropy
(100 * coinciding_stratified.loc[["BayesRisk outer", "ExcessRisk outer outer", "ExcessRisk central outer"]]).round(2)

,tiny_imagenet,imagenet_a,imagenet_r,imagenet_o
BayesRisk outer,50.0,83.22,82.23,72.21
ExcessRisk outer outer,50.0,77.11,76.52,74.46
ExcessRisk central outer,50.0,77.56,77.02,74.46
